# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [129]:
import re

class SpellCorrection:
    def __init__(self, data_path):
        self.bigram = self.bigram_train(data_path)
        self.vocab = self.vocab_collect(data_path)

    def vocab_collect(self, data_path):
        vocab = set()
        with open(data_path, 'r', encoding='ISO-8859-1') as f:
            for line in f.readlines():
                if len(line.split()) == 3:
                    freq, word1, word2 = line.split()
                    vocab.add(word1)
                    vocab.add(word2)

        return vocab

    def bigram_train(self, data_path):
        bigram_prob = {}
        with open(data_path, 'r', encoding='ISO-8859-1') as f:
            bigram = {}
            total_pairs = 0
            for line in f.readlines():
                if len(line.split()) == 3:
                    freq, word1, word2 = line.split()

                    if (word1, word2) not in bigram:
                        bigram[(word1, word2)] = 0

                    bigram[(word1, word2)] += (int)(freq)
                    total_pairs += (int)(freq)
            bigram_prob = {key: value / total_pairs for key, value in bigram.items()}

        return bigram_prob

    def edits1(self, word):
        "All edits that are one edit away from `word`. Source https://norvig.com/spell-correct.html"
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def editsK(self, word, k):
        if k < 0:
            print(f"Parametr k={k} must be >= 0")
            return set()
        if k == 0:
            return set(word)
        if k == 1:
            return self.edits1(word[0])
        else:
            edits_k = self.edits1(word[0])
            for word_k in self.edits1(word[0]):
                edits_k = edits_k.union(self.editsK([word_k], k - 1))

            return edits_k

    def known(self, words):
        return set(word for word in words if word in self.vocab)

    def candidates(self, word, K=1):
        candidat = set()
        for k in range(K + 1):
            candidat = candidat.union(self.known(self.editsK([word], k)))

        return candidat

    def correction(self, sentence, K=1, print_prob=False):
        words = re.findall('\w+', sentence.lower())
        if len(words) == 0:
            return ""

        last_word = words[0]
        correct_sentence = ""
        for word in words:
            if len(correct_sentence) > 0:
                correct_sentence += " "
            if word in self.vocab:
                correct_sentence += word
            else:
                candidates = list(self.candidates(word, K))
                if len(candidates) == 0:
                    correct_sentence += " " + word
                    continue
                probs = [self.bigram.get((last_word, candidate), 1e-10) for candidate in candidates]
                if (print_prob):
                    print(f"Try to change word={word}")
                argmax = (candidates[0], probs[0])
                for candidate, prob in zip(candidates, probs):
                    if print_prob:
                        print(f"Candidate={candidate}, prob={prob}")
                    if prob > argmax[1]:
                        argmax = (candidate, prob)
                correct_sentence += argmax[0]
            last_word = word

        return correct_sentence

In [130]:
spell_checker = SpellCorrection('./bigrams.txt')

In [131]:
spell_checker.correction('today i saw a dking species', print_prob=True)

Try to change word=dking
Candidate=ding, prob=5.416526363587943e-08
Candidate=king, prob=2.215359282707469e-06
Candidate=duking, prob=1e-10
Candidate=doing, prob=1e-10
Candidate=dying, prob=1.6339854530156962e-06
Candidate=eking, prob=1e-10


'today i saw a king species'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

Asnwer

- To solve this task, i choose bigram dataset, and train bigram based on this data.
- Weights for each edits are equal, and if you want you play with k, which stand for k recursive to combine all words which their distance is at most k

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [132]:
import time

# Evaluation code took from https://norvig.com/spell-correct.html
def spelltest(model, tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    start = time.time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = model.correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in model.vocab)
            if verbose:
                print('correction({}) => {}; expected {}'
                      .format(wrong, w, right))
    dt = time.time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [135]:
spelltest(spell_checker, Testset(open('spell-testset1.txt')))
spelltest(spell_checker, Testset(open('spell-testset2.txt')))

53% of 270 correct (3% unknown) at 4969 words per second 
56% of 400 correct (4% unknown) at 4920 words per second 


Seems data taht I took from https://norvig.com/spell-correct.html, is more related to single words, while bigram that I trained is more relate to 2 words text, hence low performance.